In [ ]:
from cng.utils import *
from ibis import _
import os
from osgeo import gdal

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")

set_secrets(con)

In [ ]:
# could use bash
# ! gdalwarp -t_srs EPSG:4326 -of XYZ /vsicurl/https://data.source.coop/cboettig/mobi/species-richness-all/mobi-species-richness.tif mobi.txt

In [ ]:
# os.unlink("mobi.xyz")

def gdal_warp(input_file, output_file, dst_srs):
    ds = gdal.Open(input_file)
    src_srs = ds.GetProjection()
    warp_options = gdal.WarpOptions(dstSRS=dst_srs)
    gdal.Warp(output_file, ds, options=warp_options)

input_file ="/vsicurl/https://data.source.coop/cboettig/mobi/species-richness-all/mobi-species-richness.tif"
output_file = "mobi.xyz"
dst_srs = 'EPSG:4326'  # WGS84

gdal_warp(input_file, output_file, dst_srs)

In [ ]:
mobi = con.read_csv("mobi.xyz", 
             delim = ' ', 
             columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'},
            table_name = 'mobi'
            )

zoom = 8
con.raw_sql(f'''
CREATE OR REPLACE TEMP TABLE t AS
SELECT Z,
       h3_latlng_to_cell_string(Y, X, {zoom}) AS h{zoom}
FROM mobi
''')
mobi_h3 = con.table("t")

In [ ]:
tracts_z8 = (
    con
    .read_parquet("s3://public-social-vulnerability/2022-tracts-h3-z8.parquet")
    .filter(_.STATE == "California") 
    .mutate(h8 = _.h8.lower())
)

In [ ]:

tracts_mobi = (
    tracts_z8
    .left_join(mobi_h3, 'h8')
    .group_by(_.COUNTY)
    .agg(richness = _.Z.mean())
    .order_by(_.richness.desc())
)  


tracts_mobi.execute()